# Datamodels Retriever Experiment

This document has the goal to show the implementation of a context retriever using Datamodels and its comparison against classical approaches

# 1. Run Classical Approaches

First it's needed to run the comparison subject, to do so we will use the script present in this folder, it's just necessary to run

```
python run_classical_retriever.py
```

This will run the retriever for each sample from the test dataset, saving the data at every 50 samples as checkpoint

# 2. Split Data for Datamodeling

Here we will be spliting the data to achieve a dev dataset containing a representative numbe of samples to each subtask
The "k" used here is 15

In [1]:
from src.utils import split_dev_set, subset_df
from src.retriever import DatamodelsRetriever
import pandas as pd

train = pd.read_csv("../../data/instruction-induction-data/processed/induce_tasks_examples.csv")
subset_df(train, 200, "task").to_csv("../../data/instruction-induction-data/processed/train.csv")


split_dev_set(
    path="../../data/instruction-induction-data/processed/train.csv",
    saving_path="../../data/instruction-induction-data/datamodels",    
    k_samples=15,
    task_column="task",
)

/home/caio.rhoden/miniconda3/envs/datamodels/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


                task                                     input  \
0  active_to_passive       The lawyer avoided the secretaries.   
1  active_to_passive      The judges supported the scientists.   
2  active_to_passive  The secretary recommended the professor.   
3  active_to_passive    The artists recommended the secretary.   
4  active_to_passive        The secretary avoided the senator.   

                                            output  possible_outputs  
0      The secretaries were avoided by the lawyer.               NaN  
1     The scientists were supported by the judges.               NaN  
2  The professor was recommended by the secretary.               NaN  
3    The secretary was recommended by the artists.               NaN  
4        The senator was avoided by the secretary.               NaN  
Saving dev set
Saving train set


## 3. Split the Collections to be trained

In [3]:
import pandas as pd
train_set = pd.read_csv("../../data/instruction-induction-data/datamodels/train_set.csv")
train_set

,task,input,output,possible_outputs
0,antonyms,wholly,partly,NaN
1,antonyms,regular,irregular,NaN
2,antonyms,mother,father,NaN
3,antonyms,separate,joint,NaN
4,antonyms,decisive,indecisive,NaN
...,...,...,...,...
3880,word_in_context,Sentence 1: He was hospitalized for extreme fa...,not the same,NaN
3881,word_in_context,Sentence 1: A cartilaginous style. Sentence 2:...,same,NaN
3882,word_in_context,Sentence 1: Do you have a light? Sentence 2: T...,not the same,NaN
3883,word_in_context,Sentence 1: Blow on the soup to cool it down. ...,not the same,NaN
